In [37]:
import numpy as np
import x
import numpy as np 

class Optimizer:
    def __init__(self, layers, optimizer_type='sgd', learning_rate=0.01, beta1=0.9, beta2=0.999, epsilon=1e-08):
        self.optimizer_type = optimizer_type
        self.learning_rate = learning_rate
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon 
        self.v = {}
        self.s = {}
        self.layers = layers

        if(optimizer_type=='adagrad'):
            # self.t = 0
            self.weights_cache = {}
            self.bias_cache = {}
            for i in range(len(self.layers)):
                weight_cache_key = f'layer{i}_to_layer{i+1}_weights_cache'
                biase_cache_key = f'layer{i}_bias'
                self.weights_cache[weight_cache_key] = 0
                self.bias_cache[biase_cache_key] = 0
            

        
        



    def update_parameters(self, neural_net, gradients):
        if self.optimizer_type == 'sgd':
            self.sgd_update(neural_net, gradients)
        elif self.optimizer_type == 'momentum':
            self.momentum_update(neural_net, gradients)
        elif self.optimizer_type == 'adagrad':
            self.adagrad_update(neural_net, gradients)
        elif self.optimizer_type == 'rmsprop':
            self.rmsprop_update(neural_net, gradients)
        elif self.optimizer_type == 'adam':
            self.adam_update(neural_net, gradients)


    def sgd_update(self,neural_net,gradients):
        for key in gradients:
            neural_net.weights[key] -= self.learning_rate*gradients[key]

    
    def momentum_update(self,neural_net,  gradients):
        for key in gradients:
            if key not in self.v:
                self.v[key] = np.zeros_like(gradients[key])

            self.v[key] = self.bet1*self.v[key]  + (1-self.beta1)*gradients[key]
            neural_net.weights -= self.learning_rate*self.v[key]


    def adagrad_update(self, neural_net, gradients,layers):

        







    def rmsprop_update(self, neural_net, gradients):
        for key in gradients:
            if key not in self.s:
                self.s[key] = np.zeros_like(gradients[key])
            self.s[key] = self.beta2 * self.s[key] + (1 - self.beta2) * (gradients[key] ** 2)
            neural_net.weights[key] -= self.learning_rate * gradients[key] / (np.sqrt(self.s[key]) + self.epsilon)


    def adam_update(self, neural_net, gradients):
        self.t += 1
        for key in gradients:
            if key not in self.v:
                self.v[key] = np.zeros_like(gradients[key])
            if key not in self.s:
                self.s[key] = np.zeros_like(gradients[key])

            self.v[key] = self.beta1 * self.v[key] + (1 - self.beta1) * gradients[key]
            self.s[key] = self.beta2 * self.s[key] + (1 - self.beta2) * (gradients[key] ** 2)

            v_corrected = self.v[key] / (1 - self.beta1 ** self.t)
            s_corrected = self.s[key] / (1 - self.beta2 ** self.t)

            neural_net.weights[key] -= self.learning_rate * v_corrected / (np.sqrt(s_corrected) + self.epsilon)



In [29]:
import numpy as np
from  optimizer import optimizer

class NeuralNetwork:
    def __init__(self, layers, optimizer_type, learning_rate, beta1, beta2, epsilon):
        self.weights = {}  # dictionary to store weight matrices
        self.biases = {}  # dictionary to store layer biases
        self.layers = layers
        self.optimizer_type = optimizer_type
        self.learning_rate = learning_rate
        self.beta1 = beta1
        self.beta2 = beta2 
        self.epsilon = epsilon

        # Initialize weights and biases
        for i in range(len(layers) - 1):
            self.weights[f'layer{i}_to_layer{i+1}_weights'] = np.random.rand(layers[i], layers[i+1])
        
        for j in range(1, len(layers)):
            self.biases[f'layer{j}_bias'] = np.zeros(layers[j])  # setting biases to 0 initially

        # Initialize optimizer
        self.optimizer = Optimizer(layers, optimizer_type, learning_rate, beta1, beta2, epsilon)


        

    def get_model_params(self):
        return self.weights,self.biases # returns weights and biases of the model

    def sigmoid(self,x):
        return 1/(1+np.exp(-x))
    
    def sigmoid_derivative(self,x):
        sig = self.sigmoid(x)
        return sig*(1-sig)
    
    def ReLU(self,x):
        return np.maximum(0,x)
    
    def ReLU_derivative(self,x):
        return np.where(x>0,1,0)
    

    def forward_pass(self,X):

        self.z_values = {} #pre-activation values
        self.layer_output = {} #post activation values 
        layer_operations = {}

        self.layer_output['layer0_output'] = X

        #forward propogation loop excluding the output layer calculations (cause of different activation functions)
        for i in range(1,len(self.biases),1): 
            weight_key = f'layer{i-1}_to_layer{i}_weights'
            bias_key = f'layer{i}_bias'
            z_key = f'z{i}'
            h_key = f'layer{i}_output'

            self.z_values[z_key] = np.dot(self.layer_output[f'layer{i-1}_output'],self.weights[weight_key]) + self.biases[bias_key] 
            self.layer_output[h_key] = self.ReLU(self.z_values[z_key])
            # print(self.layer_output[h_key])

        # calculating the output layer output using the sigmoid activation 
        self.output_layer_index = len(self.biases) 
        output_layer_index = len(self.biases) 
        weight_key = f'layer{output_layer_index-1}_to_layer{output_layer_index}_weights'
        bias_key = f'layer{output_layer_index}_bias'
        z_key = f'z{output_layer_index}'
        h_key = f'layer{output_layer_index}_output'

        self.z_values[z_key] = np.dot(self.layer_output[f'layer{output_layer_index-1}_output'], self.weights[weight_key]) + self.biases[bias_key]
        self.layer_output[h_key] = self.sigmoid(self.z_values[z_key])
        # print(self.layer_output[f'layer{output_layer_index}_output'])

        self.y_hat = self.layer_output[h_key] # final predicted output 
        


    def backward_pass(self,X,y,learning_rate):

        # creating dictionary for storing the gradients and errors
        self.gradients = {}
        self.errors = {}

        #calculating the error and gradients for the output layer function seperately (cause it has sigmoid derivative)
        self.net_error = y - self.y_hat 
        self.errors[f'layer{self.output_layer_index}_error'] = self.net_error*self.sigmoid_derivative(self.z_values[f'z{self.output_layer_index}'])
        self.gradients[f'layer{self.output_layer_index-1}_to_layer{self.output_layer_index}_weights_gradient'] = np.dot(self.layer_output[f'layer{self.output_layer_index-1}_output'].T,self.errors[f'layer{self.output_layer_index}_error'])*learning_rate


        for i in range(self.output_layer_index-1,0,-1):
            error_key = f'layer{i}_error'
            self.errors[error_key] = np.dot(self.errors[f'layer{i+1}_error'],self.weights[f'layer{i}_to_layer{i+1}_weights'].T)*self.ReLU_derivative(self.z_values[f'z{i}'])

        for j in range(self.output_layer_index-1,0,-1):
            gradient_key = f'layer{j-1}_to_layer{j}_weights_gradient'
            h_key = f'layer{j-1}_output' 
            error_key = f'layer{j}_error'
            self.gradients[gradient_key] = np.dot(self.layer_output[h_key].T,self.errors[error_key])
    


    # def adagrad(self,epsilon):




    def parameter_updates(self,learning_rate):

        #loop for parameter updates 
        for i in range(1,len(self.biases)):
            weight_key = f'layer{i-1}_to_layer{i}_weights'
            gradient_key = f'layer{i-1}_to_layer{i}_weights_gradient'
            error_key = f'layer{i}_error' 
            bias_key = f'layer{i}_bias'



            self.weights[weight_key] = self.weights[weight_key] - self.gradients[gradient_key]
            self.biases[bias_key] = self.biases[bias_key] - np.sum(self.errors[error_key],axis=0)*learning_rate

    
    def train(self, X, y, epochs, batch_size=None,learning_rate=0.01):
        for epoch in range(epochs):
            if batch_size:
                # Mini-batch gradient descent
                indices = np.random.permutation(X.shape[0])
                X_shuffled = X[indices]
                y_shuffled = y[indices]

                for i in range(0, X.shape[0], batch_size):
                    X_batch = X_shuffled[i:i + batch_size]
                    y_batch = y_shuffled[i:i + batch_size]
                    y_batch_one_hot = np.eye(np.max(y) + 1)[y_batch]

                    self.forward_pass(X_batch)
                    self.backward_pass(X_batch, y_batch_one_hot,learning_rate=0.01)
                    self.optimizer.update_parameters(self, self.gradients)  # Using optimizer to update parameters
            else:
                # Full-batch gradient descent
                y_one_hot = np.eye(np.max(y) + 1)[y]
                self.forward_pass(X)
                self.backward_pass(X, y_one_hot)
                self.optimizer.update_parameters(self, self.gradients)

            if epoch % 100 == 0:
                loss = self.compute_loss(y_one_hot)
                print(f'Epoch: {epoch}, Loss: {loss}')



    def compute_loss(self, y):
    # Compute cross-entropy loss for multi-class classification
        m = y.shape[0]  # Number of samples
        y_true_labels = np.argmax(y, axis=1)  # Get the true labels from one-hot encoding
        log_likelihood = -np.log(self.y_hat[np.arange(m), y_true_labels])
        loss = np.sum(log_likelihood) / m
        return loss

            
    def model_architecture(self):
        num_layers = len(self.layers)  # Total number of layers, including input, hidden, and output

        for i in range(num_layers):
            if i == 0:
                # Input layer
                print(f'Layer {i}: Input layer with {self.layers[i]} neurons')

            elif i == num_layers - 1:
                # Output layer
                print(f'Layer {i}: Output layer with {self.layers[i]} neurons (Sigmoid activation)')
            else:
                # Hidden layers
                print(f'Layer {i}: Hidden layer with {self.layers[i]} neurons (ReLU activation)')
        
            # Print the size of weights between this layer and the next
            if i < num_layers - 1:
                print(f'   Weights shape: {self.weights[f"layer{i}_to_layer{i+1}_weights"].shape}')
                print(f'   Biases shape: {self.biases[f"layer{i+1}_bias"].shape}')
        
        print("End of Architecture")


        
            
        



In [43]:
nn = NeuralNetwork(layers=[2,3,1],optimizer_type='adagrad',learning_rate=0.01,beta1=0.9,beta2=0.999,epsilon=1e-08)

In [44]:
nn.forward_pass(X)

In [38]:
def generate_dataset(n_samples=1000, n_features=2, n_classes=2):
    X = np.random.rand(n_samples, n_features)
    y = np.random.randint(0, n_classes, size=n_samples)
    return X, y

In [39]:
X, y = generate_dataset(n_samples=1000, n_features=2, n_classes=2)

In [45]:
nn.backward_pass(X,y,0.01) 

ValueError: shapes (1000,1000) and (1,3) not aligned: 1000 (dim 1) != 1 (dim 0)

In [42]:
X

array([[0.64243563, 0.78468864],
       [0.53175139, 0.19427961],
       [0.36872771, 0.27131184],
       ...,
       [0.06163709, 0.99141494],
       [0.69935879, 0.03260633],
       [0.37328838, 0.65743872]])